In [1]:
import pandas as pd
import altair as alt

In [2]:
starts_df = pd.read_csv("../data/skillshare_2022_starts.csv", index_col=0)
starts_df.head()

/tmp/ipykernel_127956/2526508458.py:1: DtypeWarning: Columns (24,53) have mixed types. Specify dtype option on import or set low_memory=False.
  starts_df = pd.read_csv("../data/skillshare_2022_starts.csv", index_col=0)


,id,user_uid,is_active,create_time,update_time,realized_end_time,expiration_time,first_payment_time,last_payment_time,next_billing_time,...,is_direct_to_paid,trial_length_offer,had_trial,sub_utm_source,sub_utm_campaign,sub_utm_medium,sub_utm_term,sub_utm_channel,referral_source,eligible_trial_number
0,8794519,14996631.0,False,2022-01-01 00:12:27,2022-01-01 00:12:27,2022-06-23 12:56:46,NaN,NaN,NaN,NaN,...,False,None,False,ShortUrl,teacher-referral,teacher-referral,NaN,Teacher Referral,https://www.linkedin.com/,NaN
1,8794529,23693211.0,False,2022-01-01 00:15:05,2022-01-08 18:32:43,2022-01-08 18:32:43,2022-01-08 18:32:43,2022-01-08 00:17:31,2022-01-08 00:17:31,NaN,...,False,One Week,True,NaN,NaN,NaN,NaN,Organic Search,https://www.google.com/,1.0
2,8794531,23693219.0,True,2022-01-01 00:17:15,2022-01-01 00:17:15,2022-06-23 12:56:46,2023-01-31 10:00:00,2022-01-31 11:27:12,2022-01-31 11:27:12,NaN,...,False,One Month,True,Google,Brand_EN-UK_IE,paidsearch,how much is skillshare,Paid Search (Brand),NaN,1.0
3,8794558,23693241.0,False,2022-01-01 00:25:49,2022-02-01 03:31:19,2022-02-01 03:31:19,2022-02-01 03:31:19,NaN,NaN,NaN,...,False,One Month,True,youtube,2021-04-01-peter-mckinnon,paid-influencer-video,NaN,Youtube Influencer,https://www.youtube.com/,1.0
4,8794560,11790171.0,True,2022-01-01 00:26:54,2022-01-01 00:26:59,2022-06-23 12:56:46,NaN,2022-01-01 00:26:56,2022-01-01 00:26:56,2023-01-01 00:26:55,...,False,None,False,Google,Brand_EN-US,paidsearch,skillshare,Paid Search (Brand),https://www.google.com/,NaN


In [3]:
# filter starts_df to our needed peramenters.
# only annual
starts_df = starts_df[starts_df['plan_length'] == 12]
# not B2B
starts_df = starts_df[starts_df['is_team'] == False]
# no scholarships
starts_df = starts_df[starts_df['is_scholarship'] == False]
# has a free trial
starts_df = starts_df[starts_df['is_direct_to_paid'] == False]
starts_df = starts_df[starts_df['had_trial'] == True]
# no special trial lengths
starts_df = starts_df[starts_df['trial_length_offer'].isin(['One Week', 'One Month'])]

In [4]:
# add a successful conversion column.
starts_df['success'] = 0

# set to 1 if they paid
starts_df['success'][starts_df['first_payment_time'].notnull()] = 1

# return to 0 if they got a refund.
starts_df['success'][starts_df['is_refunded']==1] = 0
starts_df.head()

/tmp/ipykernel_127956/3187398104.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  starts_df['success'][starts_df['first_payment_time'].notnull()] = 1
/tmp/ipykernel_127956/3187398104.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  starts_df['success'][starts_df['is_refunded']==1] = 0


,id,user_uid,is_active,create_time,update_time,realized_end_time,expiration_time,first_payment_time,last_payment_time,next_billing_time,...,trial_length_offer,had_trial,sub_utm_source,sub_utm_campaign,sub_utm_medium,sub_utm_term,sub_utm_channel,referral_source,eligible_trial_number,success
1,8794529,23693211.0,False,2022-01-01 00:15:05,2022-01-08 18:32:43,2022-01-08 18:32:43,2022-01-08 18:32:43,2022-01-08 00:17:31,2022-01-08 00:17:31,NaN,...,One Week,True,NaN,NaN,NaN,NaN,Organic Search,https://www.google.com/,1.0,0
2,8794531,23693219.0,True,2022-01-01 00:17:15,2022-01-01 00:17:15,2022-06-23 12:56:46,2023-01-31 10:00:00,2022-01-31 11:27:12,2022-01-31 11:27:12,NaN,...,One Month,True,Google,Brand_EN-UK_IE,paidsearch,how much is skillshare,Paid Search (Brand),NaN,1.0,1
3,8794558,23693241.0,False,2022-01-01 00:25:49,2022-02-01 03:31:19,2022-02-01 03:31:19,2022-02-01 03:31:19,NaN,NaN,NaN,...,One Month,True,youtube,2021-04-01-peter-mckinnon,paid-influencer-video,NaN,Youtube Influencer,https://www.youtube.com/,1.0,0
5,8794593,12529812.0,False,2022-01-01 00:40:01,2022-01-01 00:40:01,2022-02-01 00:40:01,NaN,NaN,NaN,NaN,...,One Month,True,ShortUrl,affiliate-referral-1941759612,affiliate-referral,NaN,Affiliate Referral,https://justcreative.com/,NaN,0
6,8794594,23673916.0,False,2022-01-01 00:40:30,2022-01-08 00:41:52,2022-01-08 00:41:52,2022-01-08 00:41:52,NaN,NaN,NaN,...,One Week,True,NaN,NaN,NaN,NaN,Organic Search,https://www.google.com/,1.0,0


In [5]:
referral_sources = starts_df.groupby("referral_source")['success']\
    .agg(['mean', 'size', 'std'])\
    .sort_values(by="size", ascending=False)

top20 = referral_sources[:20].reset_index()
top20

,referral_source,mean,size,std
0,https://www.google.com/,0.301757,72237,0.459023
1,https://www.youtube.com/,0.186030,33199,0.389136
2,https://join.skillshare.com/,0.214896,7371,0.410778
3,https://www.bing.com/,0.311342,3077,0.463118
4,https://duckduckgo.com/,0.358912,2463,0.479779
5,https://l.instagram.com/,0.194713,1551,0.396107
6,https://www.google.co.uk/,0.212585,1176,0.409311
7,android-app://com.google.android.gm/,0.154181,1148,0.361280
8,https%3A%2F%2Fwww.google.com%2F,0.274372,995,0.446422
9,https%3A%2F%2Fwww.skillshare.com%2Fsignup%3Fre...,0.174265,987,0.379530


In [6]:
top20["y1"] = top20['mean'] - (top20['std'] / 2)
top20["y2"] = top20['mean'] + (top20['std'] / 2)
top20

,referral_source,mean,size,std,y1,y2
0,https://www.google.com/,0.301757,72237,0.459023,0.072245,0.531268
1,https://www.youtube.com/,0.186030,33199,0.389136,-0.008539,0.380598
2,https://join.skillshare.com/,0.214896,7371,0.410778,0.009507,0.420285
3,https://www.bing.com/,0.311342,3077,0.463118,0.079783,0.542901
4,https://duckduckgo.com/,0.358912,2463,0.479779,0.119023,0.598801
5,https://l.instagram.com/,0.194713,1551,0.396107,-0.003341,0.392767
6,https://www.google.co.uk/,0.212585,1176,0.409311,0.007930,0.417240
7,android-app://com.google.android.gm/,0.154181,1148,0.361280,-0.026459,0.334821
8,https%3A%2F%2Fwww.google.com%2F,0.274372,995,0.446422,0.051161,0.497583
9,https%3A%2F%2Fwww.skillshare.com%2Fsignup%3Fre...,0.174265,987,0.379530,-0.015499,0.364030


In [7]:
bars = alt.Chart(
    title="P1 conversion from top 20 referral sources, ranked",
    data=top20.sort_values(by="mean"))\
    .mark_bar()\
    .encode(
        x=alt.X("referral_source"),
        y=alt.Y("mean", title="Average conversion rate"),
        color=alt.Color("size", title="Total referrals"))\
    .properties(
        width=700,
        height=600)

bars

alt.Chart(...)